In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
import xgboost as xgb
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from scipy.sparse import hstack

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
train=pd.read_csv('/content/drive/MyDrive/Perso/Etude perso/projet perso/ML_production_NLP_disater/train.csv')
test=pd.read_csv('/content/drive/MyDrive/Perso/Etude perso/projet perso/ML_production_NLP_disater/test.csv')

In [ ]:
X=train.drop(columns=['target'])
y=train['target']

In [ ]:
print(y)

0       1
1       1
2       1
3       1
4       1
       ..
7608    1
7609    1
7610    1
7611    1
7612    1
Name: target, Length: 7613, dtype: int64


In [ ]:
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.33, random_state=42)

In [ ]:
X_train.head()

,id,keyword,location,text
5836,8339,rubble,"Dallas, Tejas",Photo: postapocalypticflimflam: Prodding aroun...
30,44,NaN,NaN,The end!
1879,2700,crush,NaN,Man crush everyday ???? @CristianInspire http...
6852,9820,trauma,I rap to burn shame.,@PTSD_Chat Yes. I feel the root of that is Sha...
2673,3835,detonate,"Sharkatraz/Bindle's Cleft, PA",@AutoAmes everyone hoped we would join ISIS an...


In [ ]:
y_train.head()

5836    0
30      0
1879    0
6852    1
2673    0
Name: target, dtype: int64

In [ ]:
X_train=X_train.fillna('empty')
X_test=X_test.fillna('empty')

In [ ]:
vectorizer_keyword = CountVectorizer(max_features=200)
vectorizer_location = CountVectorizer(max_features=200)


In [ ]:
X_train_location = vectorizer_location.fit_transform(X_train['location'])
X_test_location  = vectorizer_location.transform(X_test['location'])

In [ ]:
X_train_keyword = vectorizer_keyword.fit_transform(X_train['keyword'])
X_test_keyword = vectorizer_keyword.fit_transform(X_test['keyword'])

In [ ]:
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1,2), max_features=2000)

In [ ]:
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train['text'])
X_test_tfidf= tfidf_vectorizer.transform(X_test['text'])

In [ ]:
X_train_keyword.shape

(5100, 200)

In [ ]:
X_train_total=hstack([X_train_location,X_train_keyword,X_train_tfidf]).toarray()
X_test_total=hstack([X_test_location,X_test_keyword,X_test_tfidf]).toarray()

In [ ]:
train_data=xgb.DMatrix(data=X_train_total,label=y_train)
validation_data=xgb.DMatrix(data=X_test_total,label=y_test)


# logistic regression

In [ ]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(random_state=0).fit(X_train_total, y_train)

In [ ]:
clf.score(X_test_total, y_test)

0.7433346597692002

In [ ]:
y_pred=clf.predict(X_test_total)
print(y_pred)

[0 0 1 ... 1 0 0]


In [ ]:
y_pred_dd=pd.DataFrame(y_pred)
y_pred_dd.value_counts()

0    1599
1     914
dtype: int64

In [ ]:
import joblib

In [ ]:
joblib.dump(clf,'/content/drive/MyDrive/Perso/Etude perso/projet perso/ML_production_NLP_disater/logistic_regression.pkl')

['/content/drive/MyDrive/Perso/Etude perso/projet perso/ML_production_NLP_disater/logistic_regression.pkl']

In [ ]:
joblib.dump(vectorizer_keyword,'/content/drive/MyDrive/Perso/Etude perso/projet perso/ML_production_NLP_disater/vectorizer_keyword.pkl')

['/content/drive/MyDrive/Perso/Etude perso/projet perso/ML_production_NLP_disater/vectorizer_keyword.pkl']

In [ ]:
joblib.dump(vectorizer_location,'/content/drive/MyDrive/Perso/Etude perso/projet perso/ML_production_NLP_disater/vectorizer_location.pkl')

['/content/drive/MyDrive/Perso/Etude perso/projet perso/ML_production_NLP_disater/vectorizer_location.pkl']

In [ ]:
joblib.dump(tfidf_vectorizer,'/content/drive/MyDrive/Perso/Etude perso/projet perso/ML_production_NLP_disater/tfidf_vectorizer.pkl')

['/content/drive/MyDrive/Perso/Etude perso/projet perso/ML_production_NLP_disater/tfidf_vectorizer.pkl']